<a href="https://colab.research.google.com/github/Anitalbs2/Financial-Matching-BRF/blob/main/Code_FM_BFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import numpy as np


Data Cleaning

To ensure reliability and consistency, the dataset was preprocessed by replacing missing values and removing duplicates to maintain uniqueness and prevent bias.

In [ ]:
# Removing duplicates
data.drop_duplicates(inplace=True)

# Handling missing values - assuming missing data can be replaced with the median or mode as appropriate
data.fillna({
    'totalIncome': data['totalIncome'].median(),
    'totalOutgoing': data['totalOutgoing'].median(),
    'net_savings': data['net_savings'].median(),
    'weekly_spend_food': data['weekly_spend_food'].median(),
    # Add other columns as necessary
}, inplace=True)

# Ensure no negative values in financial columns
financial_columns = ['totalIncome', 'totalOutgoing', 'net_savings', 'weekly_spend_food', 'monthly_spend_food']
for col in financial_columns:
    data[col] = data[col].clip(lower=0)

In [ ]:
# Rename 'anonymised_user_id' to sequential 'User_N'
user_ids = ['User_' + str(i+1) for i in range(len(data))]
data['anonymised_user_id'] = user_ids

# Rename 'anonymised_user_id' to sequential 'User_N'
user_ids = ['User_' + str(i+1) for i in range(len(data))]
data['anonymised_user_id'] = user_ids

# Replace NaN values with "Not Answer"
data.fillna("Not Answer", inplace=True)

Financial Ratio

Financial Metric Calculation:
•	Savings Ratio: net savings / total Income
•	Spending Ratio: tota lOutgoing / total Income
•	Debt-to-Income Ratio: debt payments / total Income



In [ ]:
def calculate_financial_ratios(data):
    columns_to_convert = ['totalIncome', 'totalOutgoing', 'net_savings', 'weekly_spend_food', 'monthly_spend_food']
    for col in columns_to_convert:
        data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0)

    # Savings Ratio
    data['savings_ratio'] = data['net_savings'] / data['totalIncome']
    data['savings_ratio'].replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinity with 0

    # Spending Ratio
    data['spending_ratio'] = data['totalOutgoing'] / data['totalIncome']
    data['spending_ratio'].replace([np.inf, -np.inf], 0, inplace=True)

    # Debt-to-Income Ratio
    data['debt_to_income_ratio'] = data['totalOutgoing'] / data['totalIncome']
    data['debt_to_income_ratio'].replace([np.inf, -np.inf], 0, inplace=True)

    # Weekly Spend to Income Ratio
    data['weekly_spend_income_ratio'] = data['weekly_spend_food'] / data['totalIncome']
    data['weekly_spend_income_ratio'].replace([np.inf, -np.inf], 0, inplace=True)

    # Monthly Spend to Income Ratio
    data['monthly_spend_income_ratio'] = data['monthly_spend_food'] / data['totalIncome']
    data['monthly_spend_income_ratio'].replace([np.inf, -np.inf], 0, inplace=True)

    # Fill NaN with 0 after calculations
    data.fillna(0, inplace=True)

User Segmentation

1. User segmentation function based on financial ratios, categorizing users into different groups for savings, spending, and debt-to-income ratios.

In [ ]:
# User Segmentation Based on Financial Ratios
def segment_users(data):
    # Segment Users Based on Savings Ratio
    data['Savings Segment'] = pd.cut(
        data['savings_ratio'],
        bins=[-np.inf, 0.1, 0.3, 0.5, np.inf],
        labels=['Very Low Saver', 'Low Saver', 'Moderate Saver', 'High Saver']
    )

    # Segment Users Based on Spending Ratio
    data['Spending Segment'] = pd.cut(
        data['spending_ratio'],
        bins=[-np.inf, 0.3, 0.6, 0.8, np.inf],
        labels=['Controlled Spender', 'Moderate Spender', 'High Spender', 'Over Spender']
    )

    # Segment Users Based on Debt-to-Income Ratio
    data['Debt Segment'] = pd.cut(
        data['debt_to_income_ratio'],
        bins=[-np.inf, 0.3, 0.5, 0.7, np.inf],
        labels=['Low Debt', 'Moderate Debt', 'High Debt', 'Very High Debt']

2. Visualization of user segments based on savings, spending, and debt-to-income ratios using bar charts to display the distribution of each financial category.

In [ ]:
# Plot User Segments
def plot_user_segments(data):
    plt.figure(figsize=(14, 7))

    # Savings Segment Distribution
    plt.subplot(131)
    sns.countplot(x='Savings Segment', data=data, palette='Blues')
    plt.title('Savings Segment Distribution')
    plt.xlabel('Savings Segment')
    plt.ylabel('Count')

    # Spending Segment Distribution
    plt.subplot(132)
    sns.countplot(x='Spending Segment', data=data, palette='Greens')
    plt.title('Spending Segment Distribution')
    plt.xlabel('Spending Segment')
    plt.ylabel('Count')

    # Debt Segment Distribution
    plt.subplot(133)
    sns.countplot(x='Debt Segment', data=data, palette='Reds')
    plt.title('Debt Segment Distribution')
    plt.xlabel('Debt Segment')
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()


3. Generating personalized financial recommendations based on user segmentation in savings, spending, and debt categories, illustrating how to apply specific advice based on each user's financial behavior.

In [ ]:
# Generate Recommendations Based on User Segments
def generate_segmented_recommendations(user):
    recommendations = []

    # Savings Segment Recommendations
    if user['Savings Segment'] == 'Very Low Saver':
        recommendations.append('Consider saving at least 5% of your income each month.')
    elif user['Savings Segment'] == 'Low Saver':
        recommendations.append('Try increasing your savings gradually.')

    # Spending Segment Recommendations
    if user['Spending Segment'] == 'Over Spender':
        recommendations.append('Reduce unnecessary expenses to maintain financial stability.')
    elif user['Spending Segment'] == 'High Spender':
        recommendations.append('Monitor your monthly spending more closely.')

    # Debt Segment Recommendations
    if user['Debt Segment'] == 'High Debt':
        recommendations.append('Prioritize paying off your debts to reduce financial burden.')
    elif user['Debt Segment'] == 'Very High Debt':
        recommendations.append('Seek financial advice to manage your debt efficiently.')

    return recommendations

# User Segmentation (Ensure this is called before generating recommendations)
data = segment_users(data)



Matching System

Design of a matching system that pairs users based on financial habits and needs, facilitating targeted advice and support for improved financial wellness

In [ ]:
 Generate Recommendations for All Users
def generate_recommendations_for_all(data):
    recommendations_list = []

    for _, user in data.iterrows():
        recommendations = []

        # Savings Ratio Recommendation
        if user['savings_ratio'] < 0.1:
            recommendations.append('Increase your savings to at least 10% of your income.')
        else:
            recommendations.append('Continue good savings habits.')

        # Spending Ratio Recommendation
        if user['spending_ratio'] > 0.6:
            recommendations.append('Reduce unnecessary spending to stay within 60% of your income.')
        else:
            recommendations.append('Your spending habits are well-controlled.')

        # Debt Ratio Recommendation
        if user['debt_to_income_ratio'] > 0.2:
            recommendations.append('Prioritize paying off debt to keep it below 20% of your income.')
        else:
            recommendations.append('You have good debt management skills.')

        recommendations_list.append({
            'User ID': user.get('anonymised_user_id', user.name),
            'Savings Ratio': user['savings_ratio'],
            'Spending Ratio': user['spending_ratio'],
            'Debt-to-Income Ratio': user['debt_to_income_ratio'],
            'Financial Health': 'Moderate' if user['debt_to_income_ratio'] > 0.2 else 'Good',
            'User Segment': 'Struggling' if user['savings_ratio'] < 0.1 or user['spending_ratio'] > 0.6 else 'Secure',
            'Recommendations': '; '.join(recommendations)
        })

    recommendations_df = pd.DataFrame(recommendations_list)
    return recommendations_df


In [ ]:
# Display Recommendations Table
def display_recommendations_table(data):
    recommendations_df = generate_recommendations_for_all(data)

    # Display the table in a user-friendly format
    print("Financial Recommendations Table:**")
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks
    print(recommendations_df.head(10))  # Display the first 10 rows